# Part 1: EDA and feature selection
## Imports

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import scipy
from tqdm.auto import tqdm

In [2]:
!ls /kaggle/input

media-campaign-cost-prediction	playground-series-s3e11


In [3]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

## Data ingest

In [4]:
train_df = pd.read_csv('/kaggle/input/playground-series-s3e11/train.csv', index_col='id')
test_df = pd.read_csv('/kaggle/input/playground-series-s3e11/test.csv', index_col='id')
original_df = pd.read_csv('/kaggle/input/media-campaign-cost-prediction/train_dataset.csv')

In [5]:
(train_df.columns == original_df.columns).all()

True

Train has a "cost" column that is our y-values, test does not (we have to predict the cost). "Original" is much smaller than train, which was generated from it, but is the "real" ground truth.

In [ ]:
train_df['log_cost'] = np.log(train_df['cost'])
original_df['log_cost'] = np.log(original_df['cost'])

We want to fit the log_cost, not the cost, as the kaggle accuracy is MSE of log cost.

## Initial EDA

In [ ]:
original_df.nunique()

So we actually have only two numerical features, store_sales and gross_weight. The rest are ordinal or categorical.

In [ ]:
sns.heatmap(train_df.corr().abs())

Looks like "salad_bar" and "prepared_food" have a very high correlation with each other -- could be merged into one variable. They're both boolean, so we could just take the mean of the two columns. But is this a good idea? Multicollinearity is not actually bad for many models. 

To a lesser degree, store_sales and unit_sales, and num_children_at_home and total_children, and the various boolean features (coffee_bar, video_store, salar_bar, prepared_food, and florist) form correlated chunks. If the multicollinearity does prove to be an issue, could use PCA to get rid of it -- PCA produces uncorrelated variables.

## Feature importances attempt #1

scikit-learn docs suggest using a Random Forest to make an initial guess at the feature importances. Let's try that, on the original_df dataset as the generated dataset (train_df) is too large for a quick fit. Note that this is without using a test set.

In [ ]:
from sklearn.ensemble import RandomForestRegressor
forest = RandomForestRegressor(random_state=0)

In [ ]:
forest.fit(X=original_df.iloc[:(num:=None)].drop(['cost', 'log_cost'], axis=1), y=original_df.iloc[:num]['log_cost'])

In [ ]:
feature_importances = pd.concat((pd.Series(forest.feature_names_in_), pd.Series(forest.feature_importances_)), axis=1)
feature_importances.columns = ['feature', 'importance']
sns.barplot(feature_importances, x='feature', y='importance')
plt.xticks(rotation=90)
None

Hmmm... The "most importance features" look decidedly different from the ones in the top-scoring notebook.

In [ ]:
#most_important_features = ['total_children', 'num_children_at_home',
#                           'avg_cars_at home(approx).1', 'store_sqft',
#                           'coffee_bar', 'video_store', 'salad', 
#                           'florist']
#and also: store_sales(in millions)

Why the discrepancy? I already know the standardisation isn't important for a random forest, at least. Maybe we can try investigating this in more detail. Perhaps the multicollinearity sabotages the feature importances. Seems like it is important to select features by contribution when used with a test set/cross-validation, rather than just by training set feature importances? But a discussion post suggests computing the mutual information and notes that works well as well.

## Compute the mutual information of each feature with the target

In [ ]:
from sklearn.feature_selection import mutual_info_regression

In [ ]:
train_df['salad_food'] = train_df['salad_bar'] + train_df['prepared_food']
test_df['salad_food'] = test_df['salad_bar'] + test_df['prepared_food']
original_df['salad_food'] = original_df['salad_bar'] + original_df['prepared_food']

train_df = train_df.drop(['salad_bar', 'prepared_food'], axis=1)
test_df = test_df.drop(['salad_bar', 'prepared_food'], axis=1)
original_df = original_df.drop(['salad_bar', 'prepared_food'], axis=1)
None

In [ ]:
mi_reg = mutual_info_regression(original_df.drop(['log_cost', 'cost'], axis=1), original_df['cost'])

In [ ]:
mis = pd.DataFrame()
mis['feature'] = original_df.drop(['log_cost', 'cost'], axis=1).columns
mis['mi'] = mi_reg

In [ ]:
sns.barplot(mis, x='feature', y='mi')
plt.xticks(rotation=90)
None

In [ ]:
high_mi_features = [
    'unit_sales(in_millions)',
    'total_children',
    'num_children_at_home',
    'avg_cars_at home(approx).1',
    'store_sqft',
    'coffee_bar',
    'video_store',
    'salad_food',
    'florist',
]

That looks more like our correct feature importances.

## Plot the distribution of each feature

In [ ]:
fig, axes = plt.subplots(4, 4, figsize=(5.6*5, 4.8*4))
for ax, column in zip(axes.flatten(), list(original_df.columns)):
    sns.histplot(original_df[column], bins=64, ax=ax)
    ax.set_title(column)

So, that gives us a neat hypothesis for why LOFO maybe fails on gross_weight.

## Perform further feature pruning using LOFO and cross-validation

In [ ]:
lofo_scores = mis.drop('mi', axis=1).copy()

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor

In [ ]:
forest = RandomForestRegressor(random_state=0)
for idx, row in tqdm(lofo_scores.iterrows(), total=len(lofo_scores)):
    lofo_scores.at[idx, 'cv_score_without'] = cross_val_score(
        forest, 
        original_df.drop([row['feature'], 'log_cost', 'cost'], axis=1),
        original_df['log_cost'],
        cv=5,
        n_jobs=8).mean()
    print(idx/len(lofo_scores*100))

In [ ]:
baseline = cross_val_score(
    forest, 
    original_df.drop([row['feature'], 'log_cost', 'cost'], axis=1),
    original_df['log_cost'],
    cv=5,
    n_jobs=8).mean()

In [ ]:
lofo_scores['lofo_score'] = lofo_scores['cv_score_without'] - baseline

In [ ]:
sns.barplot(lofo_scores, x='feature', y='cv_score_without')
plt.xticks(rotation=90)
None

Hmmm... Honestly I'm inclined to just go by the MIs.

## Analyse duplicates

Our chosen features are 8 categorical features and maybe one numerical feature. Leaving the numerical feature out, we're left with ~3k groups, instead of 50k or 600k. So, we can greatly accelerate our model training, for our convenience.

In [ ]:
high_mi_features = [
    'unit_sales(in millions)',
    'total_children',
    'num_children_at_home',
    'avg_cars_at home(approx).1',
    'store_sqft',
    'coffee_bar',
    'video_store',
    'salad_food',
    'florist',
]

In [ ]:
train_df[high_mi_features].value_counts()

In [ ]:
train_df[high_mi_features].drop('unit_sales(in millions)', axis=1).value_counts()

In [ ]:
pd.concat((train_df, original_df))[high_mi_features].drop('unit_sales(in millions)', axis=1).value_counts()

# Part 2: Just submit something using a naive model

## Imports

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import scipy
from tqdm.auto import tqdm

In [ ]:
!ls /kaggle/input

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

## Data preparation

In [ ]:
train_df = pd.read_csv('/kaggle/input/playground-series-s3e11/train.csv', index_col='id')
test_df = pd.read_csv('/kaggle/input/playground-series-s3e11/test.csv', index_col='id')
original_df = pd.read_csv('/kaggle/input/media-campaign-cost-prediction/train_dataset.csv')

In [ ]:
train_df['log_cost'] = np.log(train_df['cost'])
original_df['log_cost'] = np.log(original_df['cost'])

In [ ]:
train_df['salad_food'] = train_df['salad_bar'] + train_df['prepared_food']
test_df['salad_food'] = test_df['salad_bar'] + test_df['prepared_food']
original_df['salad_food'] = original_df['salad_bar'] + original_df['prepared_food']

train_df = train_df.drop(['salad_bar', 'prepared_food'], axis=1)
test_df = test_df.drop(['salad_bar', 'prepared_food'], axis=1)
original_df = original_df.drop(['salad_bar', 'prepared_food'], axis=1)
None

In [ ]:
high_mi_features = [
    'unit_sales(in millions)',
    'total_children',
    'num_children_at_home',
    'avg_cars_at home(approx).1',
    'store_sqft',
    'coffee_bar',
    'video_store',
    'salad_food',
    'florist',
]
high_mi_cat_features = high_mi_features[1:]

In [ ]:
train_and_original_df = pd.concat((train_df, original_df))

In [ ]:
train_and_original_df_dedup = train_and_original_df.groupby(high_mi_features)['log_cost']\
    .agg(['mean', 'count']).sort_values('count', ascending=False).reset_index()\
    .rename(columns={'mean': 'log_cost'})

In [ ]:
train_and_original_df_dedup

## Model training

In [ ]:
from xgboost import XGBRegressor
xgbr = XGBRegressor()
xgbr.fit(X=train_and_original_df_dedup[high_mi_cat_features],
          y=train_and_original_df_dedup['log_cost'],
          sample_weight=train_and_original_df_dedup['count'])

In [ ]:
test_df['cost'] = np.expm1(xgbr.predict(test_df[high_mi_cat_features]))

In [ ]:
test_df['cost'].to_csv('/kaggle/working/predict.csv')

Wooooo! 284/953!

# Part 3: Use grid-search CV to produce an optimised XGBoost model

In [ ]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from xgboost import XGBRegressor

In [ ]:
xgb_params = {'learning_rate': 0.05,
              'tree_method': 'hist',
              'enable_categorical': True,
              'verbosity': 1,
              'random_state': 1,
              'eval_metric': 'rmse'}
xgb_tuned_params = {
    'n_estimators': np.linspace(20, 1000, 100).astype(int),
    'max_depth': np.linspace(1, 20, 100).astype(int),
    'min_child_weight': np.linspace(0, 5, 100).astype(int),
}
xgbr = XGBRegressor(**xgb_params)
cv = RandomizedSearchCV(xgbr,
                        xgb_tuned_params,
                        n_jobs=8,
                        n_iter=20,
                        verbose=3,
                        scoring='neg_root_mean_squared_error')

In [ ]:
cv.fit(X=train_and_original_df_dedup[high_mi_cat_features],
       y=train_and_original_df_dedup['log_cost'],
       sample_weight=train_and_original_df_dedup['count'])

In [ ]:
test_df['cost'] = np.expm1(cv.best_estimator_.predict(test_df[high_mi_cat_features]))
test_df['cost'].to_csv('/kaggle/working/predict_hyperparams.csv')

In [ ]:
cv.best_score_, cv.best_params_

214/953!

## Part 4: Try a LightGBM regressor instead

In [ ]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from lightgbm import LGBMRegressor

In [ ]:
lgbm_params = {'learning_rate': 0.1,
               'tree_method': 'hist',
               'random_state': 1,
               'eval_metric': 'rmse',
               'categorical_feature': [high_mi_cat_features.index('store_sqft')],
               'verbose': -1}
lgbm_tuned_params = {
    'n_estimators': np.linspace(20, 1000, 100).astype(int),
    'num_leaves': np.linspace(20, 1000, 100).astype(int),
    'min_child_weight': np.linspace(0, 5, 100).astype(int),
}
lgbmr = LGBMRegressor(**lgbm_params)
cv_lgbm = RandomizedSearchCV(lgbmr,
                             lgbm_tuned_params,
                             n_jobs=8,
                             n_iter=20,
                             verbose=3,
                             scoring='neg_root_mean_squared_error')

In [ ]:
cv_lgbm.fit(X=train_and_original_df_dedup[high_mi_cat_features],
       y=train_and_original_df_dedup['log_cost'],
       sample_weight=train_and_original_df_dedup['count'])

In [ ]:
test_df['cost'] = np.expm1(cv_lgbm.best_estimator_.predict(test_df[high_mi_cat_features]))
test_df['cost'].to_csv('/kaggle/working/predict_lgbm.csv')

In [ ]:
cv_lgbm.best_score_, cv_lgbm.best_params_

Position: 225/953... What about an ensemble?

## Part 5: Try an ensemble?

In [ ]:
train_and_original_df_dedup['xgb_pred_log_cost'] = cv.best_estimator_.predict(train_and_original_df_dedup[high_mi_cat_features])
train_and_original_df_dedup['lgbm_pred_log_cost'] = cv_lgbm.best_estimator_.predict(train_and_original_df_dedup[high_mi_cat_features])

In [ ]:
from sklearn.linear_model import Ridge

In [ ]:
rr = Ridge()
rr.fit(X=train_and_original_df_dedup[['xgb_pred_log_cost', 'lgbm_pred_log_cost']],
       y=train_and_original_df_dedup['log_cost'],
       sample_weight=train_and_original_df_dedup['count'])

In [ ]:
test_df['xgb_pred_log_cost'] = cv.best_estimator_.predict(test_df[high_mi_cat_features])
test_df['lgbm_pred_log_cost'] = cv_lgbm.best_estimator_.predict(test_df[high_mi_cat_features])

In [ ]:
#test_df['cost'] = np.expm1(rr.predict(test_df[['xgb_pred_log_cost', 'lgbm_pred_log_cost']]))
test_df['cost'] = np.expm1(np.mean(test_df[['xgb_pred_log_cost', 'lgbm_pred_log_cost']], axis=1))
test_df['cost'].to_csv('/kaggle/working/predict_ensemble_unweighted.csv')

Hmmm... Looks like ensembling with the LightGBM, at least, worsens performance on test...